# **Account Validation**

In [1]:
!pip install -q kaggle    # Install the Kaggle CLI

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# **Import Libraries**

In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# **Download Dataset from Kaggle and UnZip it.**

In [5]:
!kaggle datasets download -d mlg-ulb/creditcardfraud

Dataset URL: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
License(s): DbCL-1.0


In [6]:
#Unzipping the dataset file
import zipfile
zip_ref = zipfile.ZipFile('/content/creditcardfraud.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

# **Prprocess Dataset**

In [7]:
df=pd.read_csv('/content/creditcard.csv')

In [8]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [10]:
df.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [11]:
# Distribution of legit transactions & fraud transactions
df['Class'].value_counts()

,count
Class,
0,284315
1,492


Dataset is highly imbalanced

*   0 --> Normal Transaction
*   1 --> Fraud Transaction

In [12]:
legit=df[df.Class==0]
fraud=df[df.Class==1]

In [16]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [17]:
legit.Amount.describe()

,Amount
count,284315.000000
mean,88.291022
std,250.105092
min,0.000000
25%,5.650000
50%,22.000000
75%,77.050000
max,25691.160000


In [18]:
fraud.Amount.describe()

,Amount
count,492.000000
mean,122.211321
std,256.683288
min,0.000000
25%,1.000000
50%,9.250000
75%,105.890000
max,2125.870000


In [20]:
df.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


### **Undersampling**

In [21]:
legit_sample=legit.sample(n=492)

In [22]:
#Concatenate two dataframes
new_df=pd.concat([legit_sample, fraud], axis=0)

In [27]:
new_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
74640,55680.0,-1.522195,-2.252879,0.279877,-2.132116,-2.660857,0.621678,2.461624,-0.215988,-2.720401,...,0.298007,-0.063007,1.338021,-0.113376,0.667776,-0.237167,-0.192942,0.117699,724.77,0
83769,60020.0,-1.162482,1.532422,-0.473604,0.969604,-0.292981,-1.147027,0.707814,0.283220,-0.499515,...,0.127749,0.597590,-0.076851,0.417374,-0.745626,-0.448068,0.175596,-0.115602,75.45,0
229646,146002.0,1.948011,-1.749172,-2.942443,-1.487330,-0.235491,-1.224372,0.354907,-0.679777,-2.100118,...,0.241698,0.636351,-0.371428,0.540203,0.614452,0.258149,-0.124120,-0.040361,266.80,0
17449,28724.0,-2.880840,2.573922,-4.266041,0.995185,0.609540,-0.699215,1.006877,0.200640,2.429079,...,-0.876773,-0.644677,-0.337848,-1.731531,-0.032885,-0.432339,-0.391724,-0.833821,89.99,0
268540,163265.0,-1.461823,1.481336,0.422808,-1.001051,0.191778,1.914485,-2.169858,-4.801358,-0.320101,...,-2.322232,1.586699,0.124428,-0.179281,-0.363338,0.524464,0.307383,0.276320,9.20,0


In [28]:
new_df.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
281674,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,...,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


In [29]:
new_df['Class'].value_counts()

,count
Class,
0,492
1,492


In [30]:
new_df.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,92843.930894,0.027645,-0.002194,-0.016710,-0.004326,-0.015038,0.011236,0.016706,-0.022300,0.066967,...,-0.055193,0.007078,0.003796,-0.031349,-0.025536,-0.027913,0.004862,-0.017186,-0.009000,85.567398
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


### **Train Test Split**

In [31]:
X=new_df.drop(columns='Class', axis=1)
y= new_df['Class']

In [32]:
X

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
74640,55680.0,-1.522195,-2.252879,0.279877,-2.132116,-2.660857,0.621678,2.461624,-0.215988,-2.720401,...,1.104138,0.298007,-0.063007,1.338021,-0.113376,0.667776,-0.237167,-0.192942,0.117699,724.77
83769,60020.0,-1.162482,1.532422,-0.473604,0.969604,-0.292981,-1.147027,0.707814,0.283220,-0.499515,...,-0.078416,0.127749,0.597590,-0.076851,0.417374,-0.745626,-0.448068,0.175596,-0.115602,75.45
229646,146002.0,1.948011,-1.749172,-2.942443,-1.487330,-0.235491,-1.224372,0.354907,-0.679777,-2.100118,...,0.064918,0.241698,0.636351,-0.371428,0.540203,0.614452,0.258149,-0.124120,-0.040361,266.80
17449,28724.0,-2.880840,2.573922,-4.266041,0.995185,0.609540,-0.699215,1.006877,0.200640,2.429079,...,0.989547,-0.876773,-0.644677,-0.337848,-1.731531,-0.032885,-0.432339,-0.391724,-0.833821,89.99
268540,163265.0,-1.461823,1.481336,0.422808,-1.001051,0.191778,1.914485,-2.169858,-4.801358,-0.320101,...,1.363932,-2.322232,1.586699,0.124428,-0.179281,-0.363338,0.524464,0.307383,0.276320,9.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00


In [33]:
y

,Class
74640,0
83769,0
229646,0
17449,0
268540,0
...,...
279863,1
280143,1
280149,1
281144,1


In [35]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, stratify=y, random_state=47)

In [37]:
print(X.shape, y.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(984, 30) (984,) (787, 30) (197, 30) (787,) (197,)


# **Model Building**

In [48]:
logistic_model=LogisticRegression()
logistic_model.fit(X_train,y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

# **Model Evaluation**

In [49]:
# Model prediction on training data
y_pred=logistic_model.predict(X_train)

accuracy=accuracy_score(y_train,y_pred)
precision=precision_score(y_train, y_pred)
recall=recall_score(y_train, y_pred)
f1=f1_score(y_train,y_pred)

print(f"Accuracy:{accuracy}")
print(f"Precision:{precision}")
print(f"Recall:{recall}")
print(f"f1_score:{f1}")

Accuracy:0.9415501905972046
Precision:0.9602122015915119
Recall:0.9211195928753181
f1_score:0.9402597402597402


In [51]:
# Model prediction on test data
y_pred=logistic_model.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)
precision=precision_score(y_test, y_pred)
recall=recall_score(y_test, y_pred)
f1=f1_score(y_test,y_pred)

print(f"Accuracy:{accuracy}")
print(f"Precision:{precision}")
print(f"Recall:{recall}")
print(f"f1_score:{f1}")

Accuracy:0.9390862944162437
Precision:0.9578947368421052
Recall:0.9191919191919192
f1_score:0.9381443298969072
